# Find out average of the harvesting data

In [ ]:
import pandas as pd

# อ่านไฟล์ CSV
df = pd.read_csv('manual_harvest_data.csv')

# ลบแถวที่มีคำว่า Restock และ Empty ในคอลัมน์ Comments
df_cleaned = df[~df['Comments'].str.contains('Restock|Empty', na=False)]

# คำนวณค่าเฉลี่ยและส่วนเบี่ยงเบนมาตรฐานของคอลัมน์ Removed (kg) แยกตามแต่ละ Tank ID
grouped = df_cleaned.groupby('Tank ID')['Removed (kg)'].agg(['mean', 'std']).reset_index()
grouped.columns = ['Tank ID', 'Average Removed (kg)', 'SD Removed (kg)']

# คำนวณค่าเฉลี่ยและส่วนเบี่ยงเบนมาตรฐานของคอลัมน์ Removed (kg) โดยรวมทั้งหมด
overall_avg = df_cleaned['Removed (kg)'].mean()
overall_sd = df_cleaned['Removed (kg)'].std()

# แสดงผลลัพธ์
print("Average and SD of Removed (kg) by Tank ID:")
print(grouped)

print("\nOverall Average Removed (kg):", overall_avg)
print("Overall SD Removed (kg):", overall_sd)

# Functions to create seaweeds growth rate and generate sensor values by adding Noise

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

def generate_noise(biomass, radiation):
    sensor = []
    for i in range(biomass.shape[0]):
        if len(radiation) >= i or radiation[i] > 0:
            noise_ = np.random.normal(0, 1)
        else:
            noise_ = np.random.normal(0, 0.5)
        
        value_with_noise = biomass[i] + noise_
        value_clipped = np.clip(value_with_noise, 0, None)  # ใช้ np.clip เพื่อจำกัดค่าขั้นต่ำที่ 0
        sensor.append(value_clipped)
    
    return np.array(sensor)

def plot_graph_biomass(steps, biomass, harvest_days, dt, y_label = 'Biomass (g/L)', title='Biomass Concentration'):
    # Plot results
    plt.figure(figsize=(12, 6))
    # plt.plot(time_steps, biomass, label='Biomass Concentration')
    plt.title(title)
    plt.plot(steps, biomass, label='Biomass Concentration')

    # เพิ่มเส้นแนวตั้งสีชมพูในวันที่มีการ harvest
    for harvest_day in harvest_days:
        plt.axvline(x=harvest_day * dt, color='pink', linestyle='--')

    plt.ylim(ymin=0)
    plt.xlabel('Time (days)')
    plt.ylabel(y_label)
    plt.legend()
    plt.tight_layout()
    plt.show()

# ฟังก์ชันคำนวณการเจริญเติบโตของ seaweed
def simulate_seaweed_growth(algae_index, tank_id, initial_biomass_kg=22.11, weeks=8, is_print = False):
    # อ่านไฟล์ข้อมูล
    df = pd.read_csv('processed_weather_algae_' + str(algae_index) + '.csv')

    I_max = df['Solar Radiation W/m^2'].max()
    optimal_temp = df['Water Temperature C'].mean()
    optimal_pH = df['pH'].mean()

    if is_print:
        print(f"Max Solar Radiation W/m^2: {I_max}")
        print(f"Optimal temperature {optimal_temp}")
        print(f"Optimal pH {optimal_pH}")

    # แปลงคอลัมน์ datetime ให้เป็น datetime object
    df['datetime'] = pd.to_datetime(df['datetime'])

    # เลือกเฉพาะข้อมูลที่มี day_of_week = 1 (วันพฤหัส)
    thursday_df = df[df['day_of_week'] == 0]

    # สุ่มเลือกวันพฤหัสจากตาราง และตรวจสอบให้แน่ใจว่า start_date มีค่าน้อยกว่าวันสุดท้ายในตาราง 9 สัปดาห์
    latest_date = df['datetime'].max() - pd.Timedelta(weeks=9*7, unit='days')
    start_date = thursday_df[thursday_df['datetime'] <= latest_date]['datetime'].sample().values[0]
    end_date = start_date + pd.Timedelta(days=7 * 8, unit='days')

    # ตั้งค่าเวลา 10:00 น. ให้กับ start_date และ end_date
    start_date = pd.Timestamp(start_date).replace(hour=10, minute=0, second=0)
    end_date = pd.Timestamp(end_date).replace(hour=10, minute=0, second=0)

    # กรองข้อมูลสำหรับช่วงเวลาที่สนใจ โดยเริ่มที่เวลา 10.00 และจบที่ 10.00
    df = df[(df['datetime'] >= start_date) & (df['datetime'] <= end_date)]
    df = df.reset_index(drop=True)

    # พิมพ์ข้อมูลวันแรก เวลาแรกสุด และวันสุดท้าย ข้อมูลท้ายสุด
    if is_print:
        print(f"Start date and time: {df['datetime'].iloc[0]}")
        print(f"End date and time: {df['datetime'].iloc[-1]}")

    # กำหนดค่าเริ่มต้นและพารามิเตอร์
    volume = 5.5 * 1.5 * 1.0  # cubic meters
    initial_biomass_concentration = initial_biomass_kg / volume  # g/L
    days = (end_date - start_date).days + 1
    time_steps = np.arange(days)
    # biomass = np.zeros_like((len(df) + 1), dtype=float)
    biomass = []
    biomass.append(initial_biomass_concentration)
    ideal_biomass = []
    ideal_biomass.append(initial_biomass_concentration)

    # หา index ของวันพฤหัสทุกสัปดาห์ในช่วงเวลาที่สนใจหลัง 10 โมงเช้า
    harvest_days = []
    old_day = df['datetime'].iloc[0].day
    old_month = df['datetime'].iloc[0].month
    last_week_no_harvest = False
    weeks_counter = 0
    no_harvest_counter = 0
    for i in range(0, len(df)):
        if df['datetime'].iloc[i].weekday() == 3 and df['datetime'].iloc[i].hour >= 10:
            if df['datetime'].iloc[i].day != old_day or df['datetime'].iloc[i].month != old_month:
                weeks_counter += 1
                old_day = df['datetime'].iloc[i].day
                old_month = df['datetime'].iloc[i].month
                if (weeks_counter >= 8):
                    continue
                if random.random() > 0.05 or last_week_no_harvest or no_harvest_counter >= 2:
                    harvest_days.append(i)
                    last_week_no_harvest = False
                else:
                    last_week_no_harvest = True
                    no_harvest_counter += 1

    time_difference = (end_date - start_date).total_seconds()
    time_step_second = time_difference /(len(df) - 1)

    if is_print:
        for i in harvest_days:
            print(f"harvest day: {i} date {df['datetime'].iloc[i]}")
        print(f'Time step in second: {time_step_second}')

    # ค่า nutrient และ temperature จากตาราง
    nutrient_concentration = df['TAN'] + df['Nitrite'] + df['Nitrate']
    ideal_nutrient_concentration = nutrient_concentration.mean()
    temperature = df['Water Temperature C']
    air_temperature = df['Air Temperature C']
    radiation = df['Solar Radiation W/m^2']

    # อ่านค่าเฉลี่ยและส่วนเบี่ยงเบนมาตรฐานของการ harvest
    harvest_stats_df = pd.read_csv('manual_harvest_data.csv')
    harvest_stats_df = harvest_stats_df[(harvest_stats_df['Comments'] != 'Restock') & (harvest_stats_df['Comments'] != 'Empty')]
    harvest_stats = harvest_stats_df.groupby('Tank ID')['Removed (kg)'].agg(['mean', 'std']).reset_index()
    avg_removed = harvest_stats.loc[harvest_stats['Tank ID'] == tank_id, 'mean'].values[0]
    sd_removed = harvest_stats.loc[harvest_stats['Tank ID'] == tank_id, 'std'].values[0]

    # Model parameters
    I0 = I_max  # Incident light at its maximum (W/m^2)
    alpha = 0.08  # Total extinction coefficient (1/m)
    k1_day = 0.45  # Growth rate constant during the day (1/days)
    k1_night = 0.045  # Growth rate constant during the night (1/days)
    k2 = 0.0045  # Death rate constant (1/days)
    kh = 0.5  # Half-saturation constant (mg/L)
    Topt = optimal_temp + 273.15  # Optimal temperature (K)
    Kt = 0.0001  # Temperature-effect coefficient
    pHopt = optimal_pH  # Optimal pH
    ideal_pH = optimal_pH
    lambda_ = 0.1  # Parameter for CO2 function
    L_inf = 90.0 / volume  # Asymptotic biomass concentration (g/L)

    def f_I(I_avg):
        return 1 / (1 + np.exp(-0.0044 * I_avg))

    def f_T(T):
        return np.exp(-Kt * (T - Topt)**2)

    def f_C(C, kh):
        return C / (C + kh)

    def f_CO2(pH, pHopt):
        return 1 / (1 + np.exp(lambda_ * (pH - pHopt)))

    def VBGF(L, L_inf, k):
        return k * (L_inf - L)
    
    seconds_per_day = 24*60*60

    # Simulation
    start_time = 0
    steps = []
    steps.append(start_time)
    biomass_kg = []
    biomass_kg.append(initial_biomass_kg)
    ideal_biomass_kg = []
    ideal_biomass_kg.append(initial_biomass_kg)
    harvest_in_step = []
    harvest_in_step.append(0)

    for i in range(len(df) - 1):
        dt = time_step_second / seconds_per_day # หน่วยของเวลาเป็นวัน
        # I_avg = I0 * np.exp(-alpha * 1.0) * max(0, np.sin(2 * np.pi * (i % 24) / 24))
        I_avg = radiation[i]
        T = temperature[i] + 273.15  # แปลงจาก C เป็น K
        air_T = air_temperature[i] + 273.15
        C_nutrient = nutrient_concentration[i]
        pH = df['pH'][i]

        start_time = dt + start_time
        steps.append(start_time)

        if I_avg > 0:  # Daytime
            growth_rate = k1_day * f_I(I_avg) * f_T(T) * f_C(C_nutrient, kh) * f_CO2(pH, pHopt)
            ideal_growth_rate = k1_day * f_I(I_avg) * f_T(air_T) * f_C(ideal_nutrient_concentration, kh) * f_CO2(ideal_pH, pHopt)
        else:  # Nighttime
            growth_rate = k1_night * f_T(T) * f_C(C_nutrient, kh) * f_CO2(pH, pHopt)
            ideal_growth_rate = k1_night * f_T(air_T) * f_C(ideal_nutrient_concentration, kh) * f_CO2(ideal_pH, pHopt)

        # Apply VBGF to growth rate
        growth_rate = VBGF(biomass[i], L_inf, growth_rate)
        death_rate = k2 * biomass[i]
        next_biomass = biomass[i] + (growth_rate - death_rate) * dt
        biomass.append(next_biomass)

        ideal_growth_rate = VBGF(ideal_biomass[i], L_inf, ideal_growth_rate)
        ideal_death_rate = k2 * ideal_biomass[i]
        ideal_next_biomass = ideal_biomass[i] + (ideal_growth_rate - ideal_death_rate) * dt
        ideal_biomass.append(ideal_next_biomass)

        # Harvest biomass on specified days
        if i in harvest_days:
            harvested_amount = min(max(np.random.normal(avg_removed, sd_removed), 12), 25)
            biomass[i + 1] = max(biomass[i + 1] - harvested_amount / volume, 1)

            ideal_biomass[i + 1] = max(ideal_biomass[i + 1] - harvested_amount / volume, 1)

            harvest_in_step.append(harvested_amount)
            # print(f'harves amound {harvested_amount}')
        else:
            harvest_in_step.append(0)

        biomass_kg.append(biomass[i + 1] * volume)
        ideal_biomass_kg.append(ideal_biomass[i + 1] * volume)

    return np.array(biomass), np.array(biomass_kg), np.array(ideal_biomass), np.array(ideal_biomass_kg), radiation, harvest_days, dt, steps, volume, df, np.array(harvest_in_step)

# Example of usage

In [ ]:
# ตัวอย่างการใช้งาน
biomass, biomass_kg, ideal_biomass, ideal_biomass_kg, radiation, harvest_days, dt, steps, volume, df, harvest_in_step = simulate_seaweed_growth(1, 1, is_print=True)
plot_graph_biomass(steps, biomass, harvest_days, dt, y_label = 'Biomass (g/L)', title='Real Biomass Density')
plot_graph_biomass(steps, biomass_kg, harvest_days, dt, y_label = 'Biomass (kg)', title='Real Biomass Mass')
plot_graph_biomass(steps, ideal_biomass, harvest_days, dt, y_label = 'Biomass (g/L)', title='Ideal Biomass Density')
plot_graph_biomass(steps, ideal_biomass_kg, harvest_days, dt, y_label = 'Biomass (kg)', title='Ideal Biomass Mass')
noise_sensor = generate_noise(biomass, radiation)
noise_sensor_kg = volume * noise_sensor
plot_graph_biomass(steps, noise_sensor, harvest_days, dt, y_label = 'sensor value')
plot_graph_biomass(steps, noise_sensor_kg, harvest_days, dt, y_label = 'sensor value (kg)')

print(f"calculate biomass kg: {biomass_kg[-1]}")
print(f"calculate ideal biomass kg: {ideal_biomass_kg[-1]}")
print(f'harvest {harvest_in_step}')

# Function of create seaweed growth rate and save as csv

In [ ]:
def create_df_biomass_sensor(biomass_kg, ideal_biomass_kg, harvest_in_step, noise_sensor_kg, df, steps, file_name, is_print = False):
    # คำนวณ sensor_val
    sensor_val = noise_sensor_kg * 500

    # สร้าง DataFrame ใหม่
    df_sensor = pd.DataFrame({
        'datetime': df['datetime'],
        'sensor_val': sensor_val,
        'Air_Temp': df['Air Temperature C'],
        'Solar_radiation': df['Solar Radiation W/m^2'],
        'humidity': df['Relative Humidity % RH'],
        'Precipitation Intensity mm/h': df['Precipitation Intensity mm/h'],
        'Total Precipitation Millimeters': df['Total Precipitation Millimeters'],
        'TAN': df['TAN'],
        'Nitrite': df['Nitrite'],
        'Nitrate': df['Nitrate'],
        'Water_temp': df['Water Temperature C'],
        'pH': df['pH'],
        'takeout_kg': harvest_in_step,
        'biomass_kg': biomass_kg,
        'ideal_biomass_kg': ideal_biomass_kg
    })

    # บันทึก DataFrame เป็นไฟล์ .csv
    df_sensor.to_csv(file_name, index=False)

    if is_print:
        print(f"Data has been saved to {file_name}")

In [ ]:
from IPython.display import clear_output

for i in range(8):
    algae_index = i + 1
    for j in range(1500):
        is_print = False
        if j % 100 == 0:
            clear_output(wait=True)
            print(f"Tank number {algae_index}")
            print(f"generate steps {j + 1}")
            is_print = True
        biomass, biomass_kg, ideal_biomass, ideal_biomass_kg, radiation, harvest_days, dt, steps, volume, df, harvest_in_step = simulate_seaweed_growth(algae_index, algae_index, is_print=is_print)
        # plot_graph_biomass(steps, biomass, harvest_days, dt, y_label = 'Biomass (g/L)')
        # plot_graph_biomass(steps, biomass_kg, harvest_days, dt, y_label = 'Biomass (kg)')
        noise_sensor = generate_noise(biomass, radiation)
        noise_sensor_kg = volume * noise_sensor
        # plot_graph_biomass(steps, noise_sensor, harvest_days, dt, y_label = 'sensor value')
        # plot_graph_biomass(steps, noise_sensor_kg, harvest_days, dt, y_label = 'sensor value (kg)')
        create_df_biomass_sensor(biomass_kg, ideal_biomass_kg, harvest_in_step, noise_sensor_kg, df, steps, file_name=f'generate_sensors/tank_{algae_index}/sensor_data_{str(j+1).zfill(5)}.csv')
        if j % 10 == 0:
            print(f"\rgenerate steps {(j + 1):05}", end='', flush=True)
        

# The sensor values are too much, reduce the signal

In [ ]:
def aggregate_data(file_path, output_path):
    # อ่านข้อมูลจากไฟล์
    df = pd.read_csv(file_path, parse_dates=['datetime'])

    # ตรวจสอบให้แน่ใจว่าข้อมูลถูกจัดเรียงตาม datetime
    df = df.sort_values('datetime').reset_index(drop=True)

    # ฟังก์ชันสำหรับการเฉลี่ยค่าและหาค่า max ของ takeout_kg
    def aggregate_chunk(chunk):
        aggregated = {}
        aggregated['datetime'] = chunk['datetime'].iloc[0]  # ใช้ค่าแรกของ datetime
        aggregated['sensor_val'] = chunk['sensor_val'].mean().astype(int)
        aggregated['Air_Temp'] = round(chunk['Air_Temp'].mean(), 1)
        aggregated['Solar_radiation'] = chunk['Solar_radiation'].mean().astype(int)
        aggregated['humidity'] = chunk['humidity'].mean().astype(int)
        aggregated['Precipitation Intensity mm/h'] = round(chunk['Precipitation Intensity mm/h'].mean(), 2)
        aggregated['Total Precipitation Millimeters'] = round(chunk['Total Precipitation Millimeters'].mean(), 2)
        aggregated['TAN'] = round(chunk['TAN'].mean(), 4)
        aggregated['Nitrite'] = round(chunk['Nitrite'].mean(), 4)
        aggregated['Nitrate'] = round(chunk['Nitrate'].mean(), 4)
        aggregated['Water_temp'] = round(chunk['Water_temp'].mean(), 1)
        aggregated['pH'] = round(chunk['pH'].mean(), 4)
        aggregated['takeout_kg'] = round(chunk['takeout_kg'].max(), 2)
        aggregated['biomass_kg'] = round(chunk['biomass_kg'].min(), 2)
        aggregated['ideal_biomass_kg'] = round(chunk['ideal_biomass_kg'].min(), 2)
        return pd.Series(aggregated)

    # แบ่งข้อมูลเป็นกลุ่มๆ ละ 3 แถว แล้วนำฟังก์ชัน aggregate_chunk มาทำงาน
    aggregated_df = df.groupby(df.index // 3).apply(aggregate_chunk).reset_index(drop=True)

    # บันทึกข้อมูลที่รวมแล้วเป็นไฟล์ใหม่
    aggregated_df.to_csv(output_path, index=False)

In [ ]:
for i in range(0,8):
    algae_index = i + 1
    for j in range(1500):
        if j % 10 == 0:
            print(f"\rtank {i+1} generate steps {(j + 1):05}", end='', flush=True)
        file_name=f'generate_sensors/tank_{algae_index}/sensor_data_{str(j+1).zfill(5)}.csv'
        dst_name=f'generate_reduce_sensors/tank_{algae_index}/sensor_data_{str(j+1).zfill(5)}.csv'
        aggregate_data(file_name, dst_name)

# Find out the average and SD to use for normalizing the real sensors

In [ ]:
import pandas as pd
for i in range(8):
    algae_index = i + 1
    min_ = 99999999999
    max_ = -99999999999
    sum_ = 0
    count = 0
    sd_sum = 0
    counter = 0
    if i != 2 and i != 5:
        continue
    for j in range(1500):
        dst_name=f'generate_reduce_sensors/tank_{algae_index}/sensor_data_{str(j+1).zfill(5)}.csv'
        df = pd.read_csv(dst_name)
        val = df['sensor_val']
        if (min_ > val.min()):
            min_ = val.min()
        if (max_ < val.max()):
            max_ = val.max()
        count += len(val)
        sum_ += val.sum()
        sd_sum += val.std()
        counter += 1

    avg_ = sum_ / count
    sd_ = sd_sum / counter
    print()
    print(f'statistic of tank {algae_index}')
    print(f'Average: {avg_}')
    print(f'Count: {count}')
    print(f'Maximum: {max_}')
    print(f'Minimum: {min_}')
    print(f"sd: {sd_}")
    print()

# Additional 1 day and 3 days monitoring

In [ ]:
import pandas as pd

def one_three_days_add(dst_name):
    # Create DataFrame
    df = pd.read_csv(dst_name)

    # Convert datetime column to actual datetime objects
    df['datetime'] = pd.to_datetime(df['datetime'])

    # Initialize the new columns with 0
    df['biomass_kg_1day'] = 0
    df['biomass_kg_3day'] = 0

    # Define the logic for filling biomass_kg_1day
    df.loc[df['datetime'].dt.hour == 10, 'biomass_kg_1day'] = df['biomass_kg']

    # Define the logic for filling biomass_kg_3day
    df.loc[(df['datetime'].dt.hour == 10) & (df['datetime'].dt.day % 3 == 0), 'biomass_kg_3day'] = df['biomass_kg']

    firsthr = False
    oldday = 0
    for i in range(len(df)):
        row = df.iloc[[i]]
        hr = row['datetime'].dt.hour
        hr = hr[i]
        ddd = row['datetime'].dt.day
        ddd = ddd[i]
        biomass1 = row['biomass_kg_1day']
        biomass1 = biomass1[i]

        biomass3 = row['biomass_kg_3day']
        biomass3 = biomass3[i]

        if hr == 10 and ddd != oldday:
            # print(i, hr, ddd, biomass1, biomass3)
            oldday = ddd
            df.loc[i, 'biomass_kg_1day'] = biomass1
            df.loc[i, 'biomass_kg_3day'] = biomass3
        else:
            df.loc[i, 'biomass_kg_1day'] = 0
            df.loc[i, 'biomass_kg_3day'] = 0

    df.to_csv(dst_name)

for i in range(5,8):
    algae_index = i + 1
    if i > 5:
        x = 0
    else:
        x = 1300
    for j in range(x, 1500):
        if j % 10 == 0:
            print(f"\rtank {i+1} generate steps {(j + 1):05}", end='', flush=True)
        dst_name=f'generate_reduce_sensors/tank_{algae_index}/sensor_data_{str(j+1).zfill(5)}.csv'
        one_three_days_add(dst_name)